In [8]:
import sys
import json
import numpy as np
import random
import time
sys.path.append('../utils/')
from utils import gpt_decoder, load_json, load_jsonl
def write(fp, line):
    with open(fp, 'a', encoding='utf-8') as f:
        f.write(json.dumps(line) + '\n')
def convert_options_to_options(options):
    return '\n'.join([chr(ord('A') + i) + '. ' + o for i, o in enumerate(options)])

In [2]:
l1_datas = load_jsonl('./timedial_f1_timebench.jsonl')
l2_datas = load_jsonl('./timedial_f2_timebench.jsonl')

In [5]:
l2_datas[0], \
    l1_datas[0]

({'qid': 16,
  'context': "A:I need to get some business cards printed up .\nB: Do you know how many you need ?\nA: I think I can get by with 2,000 .\nB: We can get started as soon as you fill out this form .\nA: I hope you can make the new cards just like my old cards .\nB: You won't be able to tell the difference with a magnifying glass .\nA: ... Okay , I'm through . Here's the form and my old card .\nB: Very good . You can pick up your order in <MASK> .\nA: Now that I think of it , three days is better than seven .\nB: We can do that , but you'll have to pay a little extra .",
  'options': ['one week', 'three days', '2 months'],
  'labels': ['yes', 'no', 'no']},
 {'qid': '16_1_3',
  'context': "A:I need to get some business cards printed up .\nB: Do you know how many you need ?\nA: I think I can get by with 2,000 .\nB: We can get started as soon as you fill out this form .\nA: I hope you can make the new cards just like my old cards .\nB: You won't be able to tell the difference wit

In [6]:
print(l1_datas[0]['context'])

A:I need to get some business cards printed up .
B: Do you know how many you need ?
A: I think I can get by with 2,000 .
B: We can get started as soon as you fill out this form .
A: I hope you can make the new cards just like my old cards .
B: You won't be able to tell the difference with a magnifying glass .
A: ... Okay , I'm through . Here's the form and my old card .
B: Very good . You can pick up your order in <MASK> .
A: Now that I think of it , three days is better than seven .
B: We can do that , but you'll have to pay a little extra .


In [13]:
prompt100 = """There is a two-person dialogue with several options.
Choose all appropriate options to substitute the <mask> in the dialogue.

dialogue: {}
options: {}
answer: 
"""

In [15]:
N = 1
data = l2_datas[N]
context = data['context']
options = convert_options_to_options(data['options'])
labels = data['labels']
gpt_decoder(prompt100.format(context, options)), options, labels

(['A. fifteen PM\nB. thirty PM'],
 'A. fifteen PM\nB. thirty PM\nC. two AM\nD. four AM',
 ['yes', 'yes', 'no', 'no'])

In [14]:
prompt100.format(context, options)

'There is a two-person dialogue with several options.\nChoose all appropriate options to substitute the <mask> in the dialogue.\n\ndialogue: A:Hello , this is Francis from XYZ Company . I would like to discuss with you about a new project some time next week . Do you have time to meet ?\nB: Well , I am not sure . Lets me check my agenda\nA: Ok .\nB: I am free on Tuesday afternoon from 2 PM to 4 PM , is it convenient for you ?\nA: It is ok with me . So I will be arriving at your office around 2 <MASK> . Do you have a projector in your room ? I would like to show you some related charts\nB: Yes , there is . See you then .\noptions: A. fifteen PM\nB. thirty PM\nC. two AM\nD. four AM\nanswer: \n'

In [16]:
N = 5
data = l2_datas[N]
context = data['context']
options = convert_options_to_options(data['options'])
labels = data['labels']
gpt_decoder(prompt100.format(context, options)), options, labels

(['B. 9:00 AM to 6:00 PM'],
 "A. 10 o'clock in the morning to 2 o'clock in the afternoon\nB. 9:00 AM to 6:00 PM \nC. 12:00 AM to 2:00 AM \nD. 7:00 PM to 9:00 PM ",
 ['yes', 'yes', 'no', 'no'])

In [17]:
N = 10
data = l2_datas[N]
context = data['context']
options = convert_options_to_options(data['options'])
labels = data['labels']
gpt_decoder(prompt100.format(context, options)), options, labels

(['C. one week'],
 'A. three days\nB. two days\nC. one week\nD. an hour',
 ['yes', 'yes', 'no', 'no'])

In [18]:
N = 25
data = l2_datas[N]
context = data['context']
options = convert_options_to_options(data['options'])
labels = data['labels']
gpt_decoder(prompt100.format(context, options)), options, labels

(['A. About three days\nB. About one day'],
 'A. About three days\nB. About one day\nC. About 12 months\nD. About 2 minutes',
 ['yes', 'yes', 'no', 'no'])

In [19]:
N = 131
data = l2_datas[N]
context = data['context']
options = convert_options_to_options(data['options'])
labels = data['labels']
gpt_decoder(prompt100.format(context, options)), options, labels

(['A. 2 years\nB. 3 years'],
 'A. 2 years\nB. 3 years\nC. 5 days\nD. 90 years',
 ['yes', 'yes', 'no', 'no'])

In [29]:
import re
answer = 'A. 2 years\nB. 3 years'
pattern = r'([A-Z]\.\s)' #For multi-choices questions with a output format A. xxx, B. xxx
answers = re.findall(pattern, answer)
answers = list(set(answers)) # de-duplication
answers = [a[0] for a in answers]
answers = sorted(answers)
answers

['A', 'B']

In [24]:
answers

['r', ' ', 's', '2', 'y', '\n', '3', 'A', 'B', '.', 'a', 'e']

In [30]:
from sklearn.metrics import f1_score

In [33]:
f1_score([1,1,0,0], [1,0,1,0], average='macro'), f1_score([1,1,0,0], [1,0,1,0], average='micro')

(0.5, 0.5)

In [34]:
from utils import calculate_f1_torque
golden = [1,2,3]
pred = [1,3,5,6]
calculate_f1_torque(golden, pred)

(0.5, 0.6666666666666666, 0.5714285714285715)

In [36]:
f1_score([1,1,1,0,0,0,0], [1,0,1,0,1,1,0]), f1_score([1,1,1,0,0,0,0], [1,0,1,0,1,1,0], average='micro')

(0.5714285714285715, 0.5714285714285714)